# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902061


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/39 [00:00<?, ?it/s]

Rendering models:   5%|▌         | 2/39 [00:03<00:57,  1.55s/it]

Rendering models:  10%|█         | 4/39 [00:06<00:56,  1.62s/it]

Rendering models:  15%|█▌        | 6/39 [00:06<00:37,  1.15s/it]

Rendering models:  21%|██        | 8/39 [00:06<00:25,  1.21it/s]

...

Rendering models:  82%|████████▏ | 32/39 [00:08<00:00, 13.22it/s]

Rendering models:  87%|████████▋ | 34/39 [00:08<00:00, 13.16it/s]

Rendering models:  92%|█████████▏| 36/39 [00:08<00:00, 13.04it/s]

Rendering models:  97%|█████████▋| 38/39 [00:09<00:00, 14.21it/s]

equidad1                              0.000212
Drew_Farnsworth                       0.007728
Thedunkmasta                          0.000379
Thedunkmasta                          0.001355
Thedunkmasta                          0.001478
Thedunkmasta                          0.001049
Thedunkmasta                          0.001026
Thedunkmasta                          0.000673
Thedunkmasta                          0.000707
Thedunkmasta                          0.000909
Thedunkmasta                          0.000747
ElisabethB                            0.352881
not-logged-in-1a3690080e09876c6142    0.000166
not-logged-in-ea8ed61a7274cfb6a43b    0.000660
AC01010                               0.000136
SohaKawtharani                        0.000546
not-logged-in-1124b2639e37eb70f4c6    0.000119
lam123ghjkl                           0.000191
Paddy_van_CH                          0.000117
not-logged-in-a0164d241f30411408f0    0.001253
not-logged-in-d41f38f63cdf5ae721ea    0.000466
ferlin1998   

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())